# Imports and Global Variables

In [2]:
from typing import List
from config.endpoints import MAINNET_BASE_URL
from helpers.utility import _make_get_request, _make_post_request
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json
from collections import defaultdict


In [3]:
try:
    ROOT_DIR = Path(__file__).resolve().parent
except NameError:
    ROOT_DIR = Path(os.getcwd()).resolve()

In [4]:
ROOT_DIR

WindowsPath('C:/Users/Brandyn/projects/solana_analysis_tool/notebooks')

In [5]:
# Persistent Variables

LAMPORT_SCALE = 1e9

In [6]:
load_dotenv()

True

In [7]:
# class TransactionsAPI:
#     def __init__(self, api_key: str):
#         self.base_url = MAINNET_BASE_URL
#         self.headers = {"Authorization": f"Bearer {api_key}"}
#         self.api_key_query = f"?api-key={api_key}"

#     def get_raw_transactions(
#         self, 
#         accounts: List[str], 
#         start_slot: int=None,
#         end_slot: int=None,
#         start_time: int=None,
#         end_time: int=None,
#         limit: int=None,
#         sort_order: str=None,
#         pagination_token: str=None
#         ):

#         path = "/v1/raw-transactions"
#         url = self.base_url + path + self.api_key_query
#         payload = {
#             "query": {
#                 "accounts": accounts,
#                 "startSlot": start_slot,
#                 "endSlot": end_slot,
#                 "startTime": start_time,
#                 "endTime": end_time
#             },
#             "options": {
#                 "limit": limit,
#                 "sortOrder": sort_order,
#                 "paginationToken": pagination_token
#             }
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transactions(self, transactions: List[str], commitment: str=None):
#         path = "/v0/transactions"
#         if commitment:
#             url = self.base_url + path + self.api_key_query + "?commitment=commitment"
#         else:
#             url = self.base_url + path + self.api_key_query
#         payload = {
#             "transactions": transactions
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transaction_history(
#         self, 
#         address: str, 
#         before: str='', 
#         until: str='', 
#         commitment: str='',
#         source: str='',
#         type: str=''
#         ):
        
#         path = f"/v0/addresses/{address}/transactions"
#         params = {
#             "before": before,
#             "until": until,
#             "commitment": commitment,
#             "source": source,
#             "type": type
#         }

#         url = self.base_url + path + self.api_key_query
#         print(f'url: {url}')
#         return _make_get_request(url, params=params)

In [8]:
HELIUS_API_KEY = os.getenv('HELIUS_API_KEY')
VYBE_API_KEY = os.getenv('VYBE_API_KEY')
NANSEN_API_KEY = os.getenv('NANSEN_API_KEY')


In [9]:
use_cache = True
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'

# Flipside Data (Balances over time)

Here is the Flipside Query; when moving to live app will use the Flipside API to retrieve this dynamically: https://flipsidecrypto.xyz/studio/queries/f72285d7-f3d4-4596-a9cd-8785187005a2

In [12]:
os.getcwd()

'C:\\Users\\Brandyn\\projects\\solana_analysis_tool\\notebooks'

In [13]:
# os.chdir('..')

In [14]:
balance_df = pd.read_csv('../data/solana_balance_history_test.csv').dropna(how='all')

# Strip BOMs or invisible characters from column names
balance_df.columns = balance_df.columns.str.replace('\ufeff', '', regex=False).str.strip()

# Optionally: remove rows where any column has just a BOM or is empty/whitespace
balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]

sol_mask = (
    (balance_df['MINT'] == 'So11111111111111111111111111111111111111111') &
    (balance_df['SYMBOL'].isna())
)

balance_df.loc[sol_mask, 'SYMBOL'] = 'SOL'
balance_df.loc[sol_mask, 'NAME'] = 'Solana'

balance_df['TX_ID'] = balance_df['TX_ID'].str.lower()

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Brandyn\AppData\Local\Temp\ipykernel_9112\1749373569.py:7: SyntaxWarning: invalid escape sequence '\s'
  balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]


In [15]:
# Any nan values left should be LP tokens

balance_df[balance_df['SYMBOL'].isna()]

,BLOCK_TIMESTAMP,OWNER,MINT,PRE_BALANCE,BALANCE,TX_ID,SUCCEEDED,SYMBOL,NAME
490,2024-01-27 20:02:00.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,21.284506,0.000000,ag9wy4nng7vgmgvwmbmikudfcgk9sigh9xhbgumq9lcfl8...,True,NaN,NaN
494,2024-01-27 20:00:42.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,21.284506,3ktq7dyxvuoknsnhkhd4wmgcvxgq5cdxm6zpuunfjc1tgb...,True,NaN,NaN
503,2024-01-27 19:56:38.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,38.261386,0.000000,5xblmj697ixeazt6azmd3fxx26ckyuwhma2pmqmpyacdry...,True,NaN,NaN
510,2024-01-27 19:54:37.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,38.261386,2jgmfwybccr51sgxupejvvprrhas1xyxpgtz96drrxnnmb...,True,NaN,NaN
519,2024-01-27 19:52:56.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,12.880316,0.000000,wmkk8qpz8xnk1kw1zaggersbbbegdm6prxwyidvbdcu4wv...,True,NaN,NaN
522,2024-01-27 19:51:51.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,10.000000,12.880316,t1egxx8diqh5ak4ptkny6ajru96dwe2r8yyjraqssj4cxy...,True,NaN,NaN
524,2024-01-27 19:47:10.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,10.000000,deymhggrurmjvatdnhmqauszfxhzrxftxps3e5bgtes7vj...,True,NaN,NaN
539,2024-01-12 00:29:26.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,B9uEZfxJoAsc51BZs6rNs8XtYVu5vfVrhbCuDCJ7mPEE,0.000000,5.000000,qwsnf1ioo2ucggywbzdfa1knav8vujgg7dkwgkazfyp6q6...,True,NaN,NaN
540,2024-01-11 23:02:04.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,5.711749,0.000000,wnz5j6dcgtjmigf8dxvbh72t4ks6bo3kvtsgmbzl7trjdn...,True,NaN,NaN
544,2024-01-11 22:57:58.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,5.711749,5238f2m2jwjbsqsnb5k1l8b6mj5z9d1xs9eeakwzjkpzsf...,True,NaN,NaN


In [16]:
# os.chdir('notebooks')

In [17]:
os.getcwd()

'C:\\Users\\Brandyn\\projects\\solana_analysis_tool\\notebooks'

# Helius API Calls

In [19]:
def get_all_signatures(account_address, helius_api_key, max_pages=20, limit=100):
    url = f"https://mainnet.helius-rpc.com/?api-key={helius_api_key}"
    collected = []
    before = None

    for _ in range(max_pages):
        payload = {
            "jsonrpc": "2.0",
            "id": "1",
            "method": "getSignaturesForAddress",
            "params": [account_address, {"limit": limit, **({"before": before} if before else {})}]
        }

        response = requests.post(
            url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(payload)
        )

        try:
            data = response.json()
            if "error" in data:
                print(f"❌ Error: {data['error']['message']}")
                break

            batch = data["result"]
            if not batch:
                break

            collected.extend(batch)
            before = batch[-1]["signature"]  # Move to next batch

            time.sleep(0.5)  # Avoid rate-limits

        except Exception as e:
            print(f"Exception: {e}")
            break

    return collected

In [20]:
# signatures = get_all_signatures(test_address, HELIUS_API_KEY)

# signatures_dict = {}

# for sig in signatures:
#     time = sig.get('blockTime')
#     signature_str = sig.get('signature')

#     signatures_dict[signature_str] = time

# signatures_dict

In [21]:
# signatures_array = list(signatures_dict.keys())

In [22]:
def v0_transactions_all(signatures, helius_api_key):
    import time
    url = f"https://api.helius.xyz/v0/transactions?api-key={helius_api_key}"
    headers = {"Content-Type": "application/json"}

    all_results = []
    batch_size = 100

    for i in range(0, len(signatures), batch_size):
        batch = signatures[i:i+batch_size]
        payload = json.dumps({"transactions": batch})

        response = requests.post(url, headers=headers, data=payload)

        try:
            data = response.json()
            if isinstance(data, dict) and "error" in data:
                print(f"❌ Error at batch {i // batch_size}: {data['error']}")
                continue

            all_results.extend(data)

        except Exception as e:
            print(f"❌ Exception during batch {i // batch_size}: {e}")

        time.sleep(0.3)  # Optional rate limit buffer

    return all_results


In [23]:
# signatures_array

In [24]:
# data = v0_transactions_all(signatures_array, HELIUS_API_KEY)

In [25]:
# len(data)

In [26]:
os.getcwd()

'C:\\Users\\Brandyn\\projects\\solana_analysis_tool\\notebooks'

In [27]:
# os.chdir('..')

In [28]:
def get_comprehensive_tx_history(wallet, api_key, use_cache=True):

    if use_cache:
        with open('../data/test_tx_history.json') as f:
            parsed_transaction_history = json.load(f)

        return parsed_transaction_history
    
    else:

        signatures = get_all_signatures(wallet, api_key)

        signatures_dict = {}

        for sig in signatures:
            time = sig.get('blockTime')
            signature_str = sig.get('signature')

            signatures_dict[signature_str] = time
        
        signatures_array = list(signatures_dict.keys())

        parsed_transaction_history = v0_transactions_all(signatures_array, api_key)

        with open('../data/test_tx_history.json', 'w') as f:
            json.dump(parsed_transaction_history, f)

        return parsed_transaction_history

In [29]:
parsed_transaction_history = get_comprehensive_tx_history(test_address, HELIUS_API_KEY, use_cache=use_cache)

In [30]:
len(parsed_transaction_history)

493

In [31]:
# os.chdir('notebooks')

In [32]:
os.getcwd()

'C:\\Users\\Brandyn\\projects\\solana_analysis_tool\\notebooks'

In [33]:
# from helius import TransactionsAPI
# transactions_api  = TransactionsAPI(HELIUS_API_KEY)

In [34]:
# os.getcwd()

In [35]:
# os.chdir('..')

In [36]:
# if not use_cache:
#     parsed_transaction_history = transactions_api.get_parsed_transaction_history(address=test_address)

#     with open('data/test_tx_history.json', 'w') as f:
#         json.dump(parsed_transaction_history, f)
# else:
#     with open('data/test_tx_history.json') as f:
#         parsed_transaction_history = json.load(f)

In [37]:
# os.chdir('notebooks')

In [38]:
# os.getcwd()

In [39]:
# print(json.dumps(parsed_transaction_history, indent=2))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [40]:
len(parsed_transaction_history) # each tx is an array

493

In [41]:
parsed_transaction_history[0].keys()

dict_keys(['description', 'type', 'source', 'fee', 'feePayer', 'signature', 'slot', 'timestamp', 'tokenTransfers', 'nativeTransfers', 'accountData', 'transactionError', 'instructions', 'events'])

In [42]:
# TX Id from Flipside dataset matches signatures in Helius dataset 

# signatures = []

# for ar in parsed_transaction_history:
#     signature = ar['signature'].lower()
#     if signature in balance_df['TX_ID'].unique():
#         print(f'ar: {ar}')


In [43]:
#block_time (UTC)

pd.to_datetime(parsed_transaction_history[0].get('timestamp'), unit='s')

Timestamp('2025-04-06 12:12:19')

In [44]:
# Account data

# account_data = {}

# for i in parsed_transaction_history[0].get('accountData'):
#     accounts = i['account']
#     if accounts and test_address in accounts:
#         nativeBalanceChange = i['nativeBalanceChange']
#         tokenBalanceChanges = i['tokenBalanceChanges']

#         account_data['nativeBalanceChange'] = nativeBalanceChange / LAMPORT_SCALE
#         account_data['tokenBalanceChanges'] = tokenBalanceChanges

# account_data

In [45]:
#Instruction data helper function

def get_instruction_data(tx, wallet):
    instructions_data = {}

    for i in tx.get('instructions'):
        accounts = i['accounts']
        if accounts and wallet in accounts:
            data = i['data']
            program_id = i['programId']
            instructions_data['data'] = data
            instructions_data['programId'] = program_id

    return instructions_data

In [46]:
# def summarize_token_transfer(tx, wallet):
#     net_token_flows = defaultdict(float)
#     counterparties = set()

#     for t in tx.get("tokenTransfers", []):
#         mint = t["mint"]
#         raw_amount = t["rawTokenAmount"]["tokenAmount"]
#         decimals = t["rawTokenAmount"]["decimals"]
#         amount = int(raw_amount) / (10 ** decimals) if raw_amount.isdigit() else 0

#         from_user = t["fromUserAccount"]
#         to_user = t["toUserAccount"]

#         if from_user == wallet:
#             net_token_flows[mint] -= amount
#             counterparties.add(to_user)
#         elif to_user == wallet:
#             net_token_flows[mint] += amount
#             counterparties.add(from_user)

#     return {
#         "timestamp": tx["timestamp"],
#         "signature": tx["signature"],
#         "net_token_flows": dict(net_token_flows),
#         "counterparties": list(counterparties),
#         "type": tx.get("type", "UNKNOWN"),
#         "source": tx.get("source", "UNKNOWN")
#     }


In [47]:
# def summarize_native_transfer(tx, wallet):
#     net_token_flows = defaultdict(float)
#     counterparties = set()

#     for t in tx.get("nativeTransfers", []):
#         amount = t.get("amount", 0) / LAMPORT_SCALE
#         from_user = t.get("fromUserAccount")
#         to_user = t.get("toUserAccount")

#         if from_user == wallet:
#             net_token_flows["SOL"] -= amount
#             counterparties.add(to_user)
#         elif to_user == wallet:
#             net_token_flows["SOL"] += amount
#             counterparties.add(from_user)

#     return {
#         "timestamp": tx["timestamp"],
#         "signature": tx["signature"],
#         "net_token_flows": dict(net_token_flows),
#         "counterparties": list(counterparties),
#         "type": tx.get("type", "UNKNOWN"),
#         "source": tx.get("source", "UNKNOWN")
#     }


In [48]:
# native_tx_dict = summarize_native_transfer(parsed_transaction_history[0], test_address)
# native_tx_dict

In [49]:
# token_tx_dict = summarize_token_transfer(parsed_transaction_history[0], test_address)
# token_tx_dict

In [50]:
def summarize_transaction(tx, wallet):
    from collections import defaultdict
    net_token_flows = defaultdict(float)
    counterparties = set()
    direction = None

    # Native transfers
    for t in tx.get("nativeTransfers", []):
        amount = t.get("amount", 0) / LAMPORT_SCALE
        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")

        if from_user == wallet:
            net_token_flows["SOL"] -= amount
            counterparties.add(to_user)
            direction = "sent"
        elif to_user == wallet:
            net_token_flows["SOL"] += amount
            counterparties.add(from_user)
            direction = "received"

    # Token transfers (safely using .get())
    for t in tx.get("tokenTransfers", []):
        mint = t.get("mint", "UNKNOWN")
        raw_token_info = t.get("rawTokenAmount", {})
        raw_amount = raw_token_info.get("tokenAmount", "0")
        decimals = raw_token_info.get("decimals", 0)

        try:
            amount = int(raw_amount) / (10 ** decimals) if raw_amount.isdigit() else 0
        except (ValueError, ZeroDivisionError):
            amount = 0

        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")

        if from_user == wallet:
            net_token_flows[mint] -= amount
            counterparties.add(to_user)
            direction = "sent"
        elif to_user == wallet:
            net_token_flows[mint] += amount
            counterparties.add(from_user)
            direction = "received"

    return {
        "timestamp": tx.get("timestamp"),
        "signature": tx.get("signature"),
        "net_token_flows": dict(net_token_flows),
        "counterparties": list(counterparties),
        "type": tx.get("type", "UNKNOWN"),
        "source": tx.get("source", "UNKNOWN"),
        "tx_status": int(tx.get("transactionError") is None),
        "direction": direction,
        "counterparty": list(counterparties)[0] if len(counterparties) == 1 else None
    }


## Transaction-Level Dataset

In [52]:
#Helper function to create row from helius and flipside data

def create_row(tx, wallet, balance_df):
    """
    Takes in a tx from helius data, a wallet address, and balance timeseries data from Flipside.  Returns a df row for analysis 
    """
    tx_summary = summarize_transaction(tx, wallet)
    instructions_data = get_instruction_data(tx, wallet)

    token_tx_df = pd.DataFrame([tx_summary])
    tx_status = "failed" if tx.get("transactionError") else "success"

    token_tx_df['slot'] = tx.get('slot')
    token_tx_df['tx_fee'] = tx.get('fee') / LAMPORT_SCALE
    token_tx_df['program_id'] = instructions_data.get('programId')
    token_tx_df['tx_status'] = tx_status #if no tx error, can assume it succeeded so we use binary 1 and 0 instead of just leaving nan
    token_tx_df['signature'] = token_tx_df['signature'].str.lower()

    filtered_balance_timeseries = balance_df[['PRE_BALANCE','BALANCE','SYMBOL','NAME','MINT','TX_ID']]
    filtered_balance_timeseries = filtered_balance_timeseries.rename(columns={'TX_ID':'signature'})

    combined_df = pd.merge(token_tx_df,filtered_balance_timeseries,on='signature', how='left')
    return combined_df 



In [53]:
#Testing the workflow to concat the dataset

combined_row = create_row(parsed_transaction_history[0], test_address, balance_df)
combined_row2 = create_row(parsed_transaction_history[1], test_address, balance_df)
pd.concat([combined_row,combined_row2]).drop_duplicates(subset='signature')

,timestamp,signature,net_token_flows,counterparties,type,source,tx_status,direction,counterparty,slot,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,MINT
0,1743941539,6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...,{'SOL': 1e-09},[5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr],TRANSFER,SYSTEM_PROGRAM,success,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,331670303,0.000006,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743898250,29ap1yrcx2dbjm4mxk8257g5e87by7cgxysds6liaamsxx...,{'SOL': 1e-09},[FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,331561835,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111


In [54]:
tx_level_data = pd.DataFrame()

for i in parsed_transaction_history:
    row = create_row(i, test_address, balance_df)
    tx_level_data = pd.concat([tx_level_data, row]).drop_duplicates(subset='signature')

tx_level_data

,timestamp,signature,net_token_flows,counterparties,type,source,tx_status,direction,counterparty,slot,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,MINT
0,1743941539,6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...,{'SOL': 1e-09},[5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr],TRANSFER,SYSTEM_PROGRAM,success,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,331670303,0.000006,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743898250,29ap1yrcx2dbjm4mxk8257g5e87by7cgxysds6liaamsxx...,{'SOL': 1e-09},[FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,331561835,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743898245,2d7d4ndxhzfwz6trn6pzy7pqefkrktqhqjq1zpz5myndvs...,{'SOL': 1e-09},[FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,331561822,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743896323,nprzwng7slnepcpebwjemkadd6wnuwscrgeweqsnshiznn...,{'SOL': 1e-05},[6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV],TRANSFER,SYSTEM_PROGRAM,success,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,331557009,0.000005,11111111111111111111111111111111,0.017763,0.017773,SOL,Solana,So11111111111111111111111111111111111111111
0,1743896305,yqsamwung3pc9h5ezjmtr1mudcuuajzye13bmfas2atlvc...,{'SOL': 2e-09},[GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg],TRANSFER,SYSTEM_PROGRAM,success,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,331556964,0.000005,11111111111111111111111111111111,0.017763,0.017763,SOL,Solana,So11111111111111111111111111111111111111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1699976664,2slwlw7rkrw4xsghxjaxmqidy2yaabjstn6u1h7hekud4d...,{'SOL': 0.0},[FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,229991035,0.000005,11111111111111111111111111111111,NaN,NaN,NaN,NaN,NaN
0,1699976634,3mwzl67964c2emlndhj37zqyjq6g2vfjqv1asfyhvednzi...,{'SOL': 0.0},[FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,229990970,0.000005,11111111111111111111111111111111,NaN,NaN,NaN,NaN,NaN
0,1699976603,2iqht33xr4mhcrkvrttzkqgcw16m8usvvv2xhxm2zgcgch...,"{'SOL': -0.236967847, 'So111111111111111111111...","[EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,...",SWAP,JUPITER,success,received,None,229990897,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469857,0.232884,SOL,Solana,So11111111111111111111111111111111111111111
0,1699975534,yvctdnezqofcvzv66mkasuuwk3cdz63wshxtshtuafxn52...,"{'SOL': -0.23697041700000002, 'So1111111111111...","[EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,...",TOKEN_MINT,UNKNOWN,failed,sent,None,229988531,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469862,0.469857,SOL,Solana,So11111111111111111111111111111111111111111


# Vybe Data

Think we will be able to get human-readable names for some addresses/program ids

In [57]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_accounts(use_cache=True):

    url = "https://api.vybenetwork.xyz/account/known-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_addresses = {}
    for account in data['accounts']:
        address = account.get('ownerAddress')
        address = address.lower()
        name = account.get('name')
        identified_addresses[address] = name

    return identified_addresses

In [58]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_programs(use_cache=True):

    url = "https://api.vybenetwork.xyz/program/known-program-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_programs = {}

    for account in data['programs']:
        program_id = account.get('programId')
        program_id = program_id.lower()
        name = account.get('name')
        identified_programs[program_id] = name

    return identified_programs

In [59]:
vybe_programs_map = get_vybe_identified_programs()

In [60]:
vybe_addresses_map = get_vybe_identified_accounts()

# Cross-Reference programIds and counterparties with known accounts/programs

In [62]:
vybe_programs_map = {k.lower(): v for k, v in vybe_programs_map.items()}
vybe_addresses_map = {k.lower(): v for k, v in vybe_addresses_map.items()}

# Add human-readable labels using .get() + str.lower()
tx_level_data['program_name'] = tx_level_data['program_id'].str.lower().map(lambda pid: vybe_programs_map.get(pid, 'Unknown Program'))

tx_level_data['counterparty_name'] = tx_level_data['counterparty'].astype(str).str.lower().map(lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))


In [63]:
# Labeled addresses that test wallet has interacted with

tx_level_data[tx_level_data['counterparty_name']!='Unknown Address']['counterparty_name'].unique()

array(['MEXC Exchange', 'Coinbase Exchange 1', 'Kamino Reserve 1',
       'SolBlaze Airdrop Wallet', 'Coinbase Exchange 3'], dtype=object)

In [64]:
# Labeled programs that test wallet has interacted with

tx_level_data[tx_level_data['program_name']!='Unknown Program']['program_name'].unique()

array(['Jupiter Swap V6', 'Allbridge Core', 'MPL Bubblegum',
       'PENGU Airdrop Claiming', 'Jupiter Swap V2', 'Parcl V3',
       'Kamino Program', 'Kamino Lend'], dtype=object)

In [138]:
# Saving for analysis in other notebooks

tx_level_data.to_csv('../data/test_acc_timeseries.csv', index=False)

# Nansen Data

In [67]:
# I think the API is not public yet

# Misc

In [69]:
# # helpers/utility.py
# def _make_post_request(url: str, payload: dict, headers: dict) -> dict:
#     response = requests.post(url, json=payload, headers=headers)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise ValueError(f"Error: {response.status_code}: {response.text}")

In [70]:
# # Initialize with your API key (ensure it's valid and active)
# #HELIUS_API_KEY = os.getenv("api_key")  
# tx_api = TransactionsAPI(api_key=HELIUS_API_KEY)

# tx_api

In [71]:
# # Example usage
# transactions = [""]  # Your transaction signatures
# response = tx_api.get_parsed_transactions(transactions)
# print(response)

In [72]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api

In [73]:
balances_api.get_balances("hq8fC9VkxU6rv1QvMUkGKbhFR3MxSd7nCkCGaaXx9MA")

{'tokens': [{'tokenAccount': '2nKanNe6tDM5aXc2ZNvYeVKpdw4Y26dvzS19FmPiJJ4S',
   'mint': 'JD8gUc2qfvmwxtttRviHSRZ4pbGKy5nha2vdEULwx3r5',
   'amount': 3888000000,
   'decimals': 6},
  {'tokenAccount': 'GXnj7rSQtBUWcQbeP5QEyKJyv4br64uckb8xsS1sBY11',
   'mint': 'G4hdrdhnboNgFXo1uP21VwZ9zrNbxwZDmtpZt2Mnpump',
   'amount': 12126567051,
   'decimals': 6},
  {'tokenAccount': '4yLdi6X9xLRV3eezR3raibRoeAxotJA3n29uEsRfoNHL',
   'mint': 'Em9zr2tgSmGgRbz3kxyQeRXjRi9oc13wMu6cKam4zWFW',
   'amount': 74500000,
   'decimals': 6},
  {'tokenAccount': '6XWCWkwCdbBTzK5GrtTCVRQqXaB2QGuDHqZsXEt9KbBo',
   'mint': 'EWx3xDR76YfxF3VHwTLgwU9WYrwANw3yxX5vvk8Kpump',
   'amount': 119000000,
   'decimals': 6},
  {'tokenAccount': 'HSrx5V9PUuazeuen6ZYaid2o1KUbiLwDaRmJdjBmqrJZ',
   'mint': 'HD1sDvk9jjUgRv6XmVYje7ghV3udzCZhotgxSortBdga',
   'amount': 1434339522508,
   'decimals': 9},
  {'tokenAccount': '8kuTaQgk46Wo7PCuXfZuySEP85fZRgVbvCXkAfwVmt9S',
   'mint': 'Hk82ay38uGpYgqjdXXCiATavVQwAMaBNyxsUWbapmKuL',
   'amount': 